<a href="https://colab.research.google.com/github/hl5chen/ECE1512-2022F-ProjectRepo_YiyangShi_HsuanlingChen/blob/main/Project_B_Supp/Task1/ECE1512_2e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 33.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
cd drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB/

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB/'
/content/drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB


# Load up distilled dataset and train model

In [20]:
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
import time

class myArgs:
  def __init__(self):
    self.method = 'DC'
    self.dataset = 'CIFAR10'
    self.model = 'ConvNet'
    self.ipc = 10
    self.eval_mode = 'S'
    self.num_exp = 1
    self.num_eval = 5
    self.epoch_eval_train = 300
    self.Iteration = 100
    self.lr_img = 0.1
    self.lr_net = 0.01
    self.batch_real = 256
    self.batch_train = 256
    self.init = 'real'
    self.dsa_strategy = 'None'
    self.data_path = 'data'
    self.save_path = 'result'
    self.dis_metric = 'ours'

args = myArgs()
args.outer_loop, args.inner_loop = get_loops(args.ipc)
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False


distilled_set_name = 'result/res_DC_CIFAR10_ConvNet_10ipc.pt'
dataset = torch.load(distilled_set_name)
syn_data = dataset['data']
syn_data = TensorDataset(syn_data[0][0], syn_data[0][1])

channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
net = get_network(args.model, channel, num_classes, im_size).to(args.device)


net.train()
optimizer_net = torch.optim.SGD(net.parameters(), lr=0.01)  # optimizer_img for synthetic data
optimizer_net.zero_grad()

criterion = nn.CrossEntropyLoss().to(args.device)

num_epochs = 20

trainloader = torch.utils.data.DataLoader(syn_data, batch_size=1, shuffle=True, num_workers=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, num_epochs)


start = time.time()

for i in range(num_epochs):
  loss_avg, acc_avg = epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)
  for param_group in optimizer_net.param_groups:
    print("Current learning rate is: {}".format(param_group['lr']))

  print("Epoch[{}/{}]: Loss: {:.4f}, ACC: {:.4f}".format(i+1,num_epochs, loss_avg, acc_avg))
  scheduler.step()


print('Training on synthetic CIFAR10 dataset took {} seconds'.format(time.time()-start))

testloader = torch.utils.data.DataLoader(dst_test, batch_size=args.batch_train, shuffle=True, num_workers=0)
net.eval()
loss_avg, acc_avg = epoch('test', testloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)

print("Test set: Loss: {:.4f}, ACC: {:.4f}".format(loss_avg, acc_avg))

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Current learning rate is: 0.01
Epoch[1/20]: Loss: 4.7377, ACC: 0.0800
Current learning rate is: 0.009938441702975689
Epoch[2/20]: Loss: 3.1479, ACC: 0.1500
Current learning rate is: 0.009755282581475769
Epoch[3/20]: Loss: 2.2229, ACC: 0.3000
Current learning rate is: 0.00945503262094184
Epoch[4/20]: Loss: 1.4145, ACC: 0.5000
Current learning rate is: 0.009045084971874739
Epoch[5/20]: Loss: 0.6055, ACC: 0.7400
Current learning rate is: 0.008535533905932738
Epoch[6/20]: Loss: 0.2835, ACC: 0.9500
Current learning rate is: 0.007938926261462366
Epoch[7/20]: Loss: 0.0843, ACC: 1.0000
Current learning rate is: 0.007269952498697735
Epoch[8/20]: Loss: 0.0447, ACC: 1.0000
Current learning rate is: 0.006545084971874738
Epoch[9/20]: Loss: 0.0332, ACC: 1.0000
Current learning rate is: 0.005782172325201155
Epoch[10/20]: Loss: 0.0277, ACC: 1.0000
Current learning rate is: 0.005
Epoch[11/20]: Loss: 0.0246, ACC: 1.0000
Current learning rate is: 0.004217827674798847
Epoch[12/20]: Loss: 0.0222, ACC: 1.00

In [24]:
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
import time

class myArgs:
  def __init__(self):
    self.method = 'DC'
    self.dataset = 'CIFAR10'
    self.model = 'ConvNet'
    self.ipc = 10
    self.eval_mode = 'S'
    self.num_exp = 1
    self.num_eval = 5
    self.epoch_eval_train = 300
    self.Iteration = 100
    self.lr_img = 0.1
    self.lr_net = 0.01
    self.batch_real = 256
    self.batch_train = 256
    self.init = 'real'
    self.dsa_strategy = 'None'
    self.data_path = 'data'
    self.save_path = 'result'
    self.dis_metric = 'ours'

args = myArgs()
args.outer_loop, args.inner_loop = get_loops(args.ipc)
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False


distilled_set_name = 'result_GM_CIFAR10_real/res_DC_CIFAR10_ConvNet_10ipc.pt'
dataset = torch.load(distilled_set_name)
syn_data = dataset['data']
syn_data = TensorDataset(syn_data[0][0], syn_data[0][1])

channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
net = get_network(args.model, channel, num_classes, im_size).to(args.device)


net.train()
optimizer_net = torch.optim.SGD(net.parameters(), lr=0.01)  # optimizer_img for synthetic data
optimizer_net.zero_grad()

criterion = nn.CrossEntropyLoss().to(args.device)

num_epochs = 20

trainloader = torch.utils.data.DataLoader(syn_data, batch_size=1, shuffle=True, num_workers=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, num_epochs)


start = time.time()

for i in range(num_epochs):
  loss_avg, acc_avg = epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)
  for param_group in optimizer_net.param_groups:
    print("Current learning rate is: {}".format(param_group['lr']))

  print("Epoch[{}/{}]: Loss: {:.4f}, ACC: {:.4f}".format(i+1,num_epochs, loss_avg, acc_avg))
  scheduler.step()


print('Training on synthetic CIFAR10 dataset took {} seconds'.format(time.time()-start))

testloader = torch.utils.data.DataLoader(dst_test, batch_size=args.batch_train, shuffle=True, num_workers=0)
net.eval()
loss_avg, acc_avg = epoch('test', testloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)

print("Test set: Loss: {:.4f}, ACC: {:.4f}".format(loss_avg, acc_avg))

Files already downloaded and verified
Files already downloaded and verified
Current learning rate is: 0.01
Epoch[1/20]: Loss: 4.5661, ACC: 0.1000
Current learning rate is: 0.009938441702975689
Epoch[2/20]: Loss: 3.6608, ACC: 0.1400
Current learning rate is: 0.009755282581475769
Epoch[3/20]: Loss: 2.6013, ACC: 0.2600
Current learning rate is: 0.00945503262094184
Epoch[4/20]: Loss: 2.1328, ACC: 0.3000
Current learning rate is: 0.009045084971874739
Epoch[5/20]: Loss: 1.3143, ACC: 0.5200
Current learning rate is: 0.008535533905932738
Epoch[6/20]: Loss: 0.7344, ACC: 0.7800
Current learning rate is: 0.007938926261462366
Epoch[7/20]: Loss: 0.4030, ACC: 0.8900
Current learning rate is: 0.007269952498697735
Epoch[8/20]: Loss: 0.1414, ACC: 0.9900
Current learning rate is: 0.006545084971874738
Epoch[9/20]: Loss: 0.0891, ACC: 1.0000
Current learning rate is: 0.005782172325201155
Epoch[10/20]: Loss: 0.0585, ACC: 1.0000
Current learning rate is: 0.005
Epoch[11/20]: Loss: 0.0475, ACC: 1.0000
Current 

In [25]:
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
import time

class myArgs:
  def __init__(self):
    self.method = 'DC'
    self.dataset = 'CIFAR10'
    self.model = 'ConvNet'
    self.ipc = 10
    self.eval_mode = 'S'
    self.num_exp = 1
    self.num_eval = 5
    self.epoch_eval_train = 300
    self.Iteration = 100
    self.lr_img = 0.1
    self.lr_net = 0.01
    self.batch_real = 256
    self.batch_train = 256
    self.init = 'real'
    self.dsa_strategy = 'None'
    self.data_path = 'data'
    self.save_path = 'result'
    self.dis_metric = 'ours'

args = myArgs()
args.outer_loop, args.inner_loop = get_loops(args.ipc)
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False


distilled_set_name = 'result_GM_CIFAR10_noise/res_DC_CIFAR10_ConvNet_10ipc.pt'
dataset = torch.load(distilled_set_name)
syn_data = dataset['data']
syn_data = TensorDataset(syn_data[0][0], syn_data[0][1])

channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
net = get_network(args.model, channel, num_classes, im_size).to(args.device)


net.train()
optimizer_net = torch.optim.SGD(net.parameters(), lr=0.01)  # optimizer_img for synthetic data
optimizer_net.zero_grad()

criterion = nn.CrossEntropyLoss().to(args.device)

num_epochs = 20

trainloader = torch.utils.data.DataLoader(syn_data, batch_size=1, shuffle=True, num_workers=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, num_epochs)


start = time.time()

for i in range(num_epochs):
  loss_avg, acc_avg = epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)
  for param_group in optimizer_net.param_groups:
    print("Current learning rate is: {}".format(param_group['lr']))

  print("Epoch[{}/{}]: Loss: {:.4f}, ACC: {:.4f}".format(i+1,num_epochs, loss_avg, acc_avg))
  scheduler.step()


print('Training on synthetic CIFAR10 dataset took {} seconds'.format(time.time()-start))

testloader = torch.utils.data.DataLoader(dst_test, batch_size=args.batch_train, shuffle=True, num_workers=0)
net.eval()
loss_avg, acc_avg = epoch('test', testloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)

print("Test set: Loss: {:.4f}, ACC: {:.4f}".format(loss_avg, acc_avg))

Files already downloaded and verified
Files already downloaded and verified
Current learning rate is: 0.01
Epoch[1/20]: Loss: 4.6467, ACC: 0.0800
Current learning rate is: 0.009938441702975689
Epoch[2/20]: Loss: 3.3095, ACC: 0.1600
Current learning rate is: 0.009755282581475769
Epoch[3/20]: Loss: 2.4250, ACC: 0.2500
Current learning rate is: 0.00945503262094184
Epoch[4/20]: Loss: 1.3810, ACC: 0.5500
Current learning rate is: 0.009045084971874739
Epoch[5/20]: Loss: 0.8884, ACC: 0.6600
Current learning rate is: 0.008535533905932738
Epoch[6/20]: Loss: 0.4493, ACC: 0.8600
Current learning rate is: 0.007938926261462366
Epoch[7/20]: Loss: 0.1973, ACC: 0.9900
Current learning rate is: 0.007269952498697735
Epoch[8/20]: Loss: 0.0890, ACC: 1.0000
Current learning rate is: 0.006545084971874738
Epoch[9/20]: Loss: 0.0490, ACC: 1.0000
Current learning rate is: 0.005782172325201155
Epoch[10/20]: Loss: 0.0395, ACC: 1.0000
Current learning rate is: 0.005
Epoch[11/20]: Loss: 0.0340, ACC: 1.0000
Current 